In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import os
import json
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from data import ImageData

pred_path_new = '../output/pix2seq_bbox_rand32_clean_all_bbox_600_4e-4_composite/prediction_val_clean_all_bbox.json'
image_base = '../data_new/detect/images/'
groundtruth_path = '../data_new/detect/splits/annotations/val_clean_all_bbox.json'

    
with open(pred_path_new) as h:
    predictions_new = json.load(h)
    
with open (groundtruth_path) as g:
    groundtruth = json.load(g)

/Mounts/rbg-storage1/users/urop/vincentf/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(predictions_new.keys())

dict_keys(['bbox', 'file_name'])


In [14]:
print(groundtruth['annotations'][0])

{'id': 357, 'image_id': 34, 'category_id': 2, 'segmentation': [], 'area': 5878.692, 'bbox': [353.78, 651.64, 160.62, 36.6], 'iscrowd': 0, 'attributes': {'occluded': False, 'rotation': 0.0}}


In [13]:
count = 0
groundtruthids = []
for image in groundtruth['images']:
    groundtruthids.append(image['id'])
print(groundtruthids)

[34, 66, 90, 94, 98, 108, 118, 134, 135, 144, 172, 173, 185, 200, 205, 228, 242, 257, 265, 267, 269, 301, 306, 314, 323, 324, 335, 339, 340, 379, 389, 394, 412, 437, 465, 483, 498, 501, 506, 514, 522, 537, 538, 539, 542, 561, 592, 628, 629, 631, 635, 643, 676, 683, 684, 713, 727, 740, 758, 769, 771, 800, 803, 805, 809, 810, 814, 819, 827, 843, 900, 901, 922, 923, 930, 934, 939, 941, 946, 954, 972, 975, 985, 986, 990, 996, 999, 1017, 1024, 1033, 1037, 1045, 1054, 1063, 1064, 1086, 1092, 1107, 1118, 1119, 1124, 1177, 1182, 1201, 1203, 1220, 1226, 1227, 1229, 1234, 1237, 1251, 1263, 1266, 1301, 1321, 1323, 1330, 1375, 1377, 1379, 1391, 1419, 1420, 1429, 1445, 914, 934, 965, 969, 972, 1032, 1036, 1059, 1076, 1078, 1083, 1092, 1098, 1101, 1102, 1103, 1108, 1115, 1118, 1119, 1121, 1134, 1146, 1149, 1154, 1190, 1194, 1196, 1201, 1228, 1232, 1245, 1252, 1291, 1292, 1301, 1303, 1308, 1309, 1319, 1322, 1335, 1359, 1369]


In [16]:
annotationsids = set()

for annotations in groundtruth['annotations']:
    annotationsids.add(annotations['image_id'])

In [18]:
print(annotationsids)

{1024, 514, 1032, 1033, 522, 1036, 1037, 1045, 537, 538, 539, 542, 1054, 34, 1059, 1063, 561, 1076, 1078, 1083, 1086, 66, 1092, 1098, 1101, 1102, 1103, 592, 1108, 90, 1115, 94, 1118, 1119, 1121, 98, 1124, 108, 1134, 628, 629, 118, 631, 1146, 635, 1149, 643, 134, 135, 144, 1177, 1182, 676, 1194, 683, 172, 173, 684, 1196, 1201, 1203, 185, 200, 713, 1227, 1228, 205, 1229, 1234, 1237, 727, 1245, 1251, 228, 740, 1252, 242, 758, 257, 769, 771, 265, 267, 1291, 269, 1292, 1301, 1303, 1308, 1309, 800, 803, 805, 1319, 809, 810, 1321, 1322, 301, 814, 1323, 306, 819, 1330, 1335, 314, 827, 323, 324, 843, 335, 1359, 339, 340, 1369, 1375, 1377, 1379, 1391, 379, 900, 389, 901, 394, 1419, 1420, 914, 1429, 922, 412, 930, 1445, 934, 939, 941, 946, 437, 954, 965, 972, 975, 465, 985, 986, 990, 483, 996, 999, 498, 501, 1017, 506}


###### the ground truth is
{'id': 969, 'width': 1904, 'height': 1748, 'file_name': 'ja029499i-Scheme-c1.png', 'license': 0, 'bboxes': [{'bbox': [1096.3, 1263.71, 62.86, 84.76], 'category_id': 2, 'id': 15675}, {'bbox': [1026.77, 5.71, 279.07, 233.32], 'category_id': 1, 'id': 15676}, {'bbox': [1249.65, 1132.29, 252.41, 63.81], 'category_id': 2, 'id': 15677}, {'bbox': [1522.88, 522.55, 146.89, 58.76], 'category_id': 2, 'id': 15678}, {'bbox': [1045.82, 554.24, 274.31, 120.95], 'category_id': 1, 'id': 15679}, {'bbox': [1533.37, 1561.28, 107.02, 52.46], 'category_id': 2, 'id': 15680}, {'bbox': [0.0, 1276.09, 284.79, 286.65], 'category_id': 1, 'id': 15681}, {'bbox': [1621.11, 1289.42, 271.46, 218.08], 'category_id': 1, 'id': 15682}, {'bbox': [454.33, 647.57, 278.12, 281.88], 'category_id': 1, 'id': 15683}, {'bbox': [486.72, 999.92, 94.29, 60.0], 'category_id': 2, 'id': 15684}, {'bbox': [1358.23, 1021.83, 96.2, 63.8], 'category_id': 2, 'id': 15685}, {'bbox': [1250.6, 925.64, 289.55, 63.81], 'category_id': 2, 'id': 15686}, {'bbox': [1621.11, 697.09, 282.89, 235.22], 'category_id': 1, 'id': 15687}, {'bbox': [1235.36, 361.88, 86.68, 52.37], 'category_id': 2, 'id': 15688}, {'bbox': [806.75, 392.35, 160.01, 99.99], 'category_id': 2, 'id': 15689}, {'bbox': [928.66, 1034.21, 219.07, 63.8], 'category_id': 2, 'id': 15690}, {'bbox': [1249.65, 787.56, 62.86, 50.47], 'category_id': 2, 'id': 15691}, {'bbox': [1045.82, 1501.79, 280.98, 243.79], 'category_id': 1, 'id': 15692}, {'bbox': [456.24, 1122.77, 282.88, 285.69], 'category_id': 1, 'id': 15693}, {'bbox': [365.75, 1382.75, 120.01, 57.14], 'category_id': 2, 'id': 15694}, {'bbox': [1248.1, 1382.4, 139.4, 62.4], 'category_id': 2, 'id': 15695}], 'caption': 'Scheme 1. Proposed Reaction Topology for the Simultaneous Interconnected Unicyclic Rh and Bimetallic Rh-Mn CBER Hydroformylation Reactions ', 'pdf': {'Page': 6, 'DPI': 100, 'Width': 850, 'Height': 1100, 'CaptionBB': [74, 64, 764, 88], 'ImageBB': [187, 88, 663, 525]}, 'diagram_type': 'graph'}